# Inference Time Study

In [1]:
# Add the new path
import sys
new_path2 = "/home/michele/code/michele_mmdet3d/"
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)

# NOTE and TODO:
#   - Try to create the loader/packer/preprocessor with the config file, and not manually inserting the parameters
#   - The variable "inputs" is used as a pointer, because the system uses it like this

wanna_print_in_out = True

In [2]:
############################################### Very original input ###############################################

home_dir = '/home/michele/code/'

# Inputs: a list of strings (each string is the ABSOLUTE path to the ".bin" file
inputs = [
    # home_dir+'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235389267178975.bin',
    # home_dir+'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235389968373008.bin',
    home_dir+'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723923467966083319.bin'
]

In [ ]:
############################################### LoadPointsFromFile ###############################################

from mmdet3d.datasets.transforms.loading import LoadPointsFromFile

# Initialize the loader
loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)

# Print the input to this stage
if wanna_print_in_out:
    print("\nLoadPointsFromFile input:")
    for element in inputs:
        print(element)

# For cycle to also handle lists of inputs
for i in range(len(inputs)):
    
    # Prepare the string input for the loader (needs two dictionaries, one nested into the other)
    inputs[i] = dict(
        lidar_points=dict(
            lidar_path=inputs[i]
        )
    )

    # Actual modification of the dictionary
    loader(inputs[i])

# Print the output of this stage
if wanna_print_in_out: 
    print("\nLoadPointsFromFile output:")
    for element in inputs:
        print(element)

In [ ]:
############################################### Pack3DDetInputs ###############################################

from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

# Initialize the packer
packer = Pack3DDetInputs(keys='points')

# Print the input to this stage
if wanna_print_in_out: 
    print("\nPack3DDetInputs input:")
    for element in inputs:
        print(element)

# For cycle to also handle lists of inputs
for i in range(len(inputs)):
    
    # Actual modification of the dictionary
    packer(inputs)

# Print the output of this stage
if wanna_print_in_out: 
    print("\nPack3DDetInputs output:")
    for element in inputs:
        print(element)

In [ ]:
############################################### Det3DDataPreprocessor ###############################################

from mmdet3d.models.data_preprocessors.data_preprocessor import Det3DDataPreprocessor

# Initialize the preprocessor
preprocessor = Det3DDataPreprocessor(
    voxel=True,
    voxel_layer=dict(
        max_num_points = 32,
        max_voxels = (20000,40000,),
        point_cloud_range = [-90, -28.8, -2, 166, 28.8, 5,],
        voxel_size = [0.16, 0.16, 7,]
    )
)

# Print the input to this stage
if wanna_print_in_out: 
    print("\nDet3DDataPreprocessor input:")
    for element in inputs:
        print(element)

# Create the list with the final inputs
final_inputs = []
for i in range(len(inputs)):    
    # Create a temporary dictionary to be passed to the preprocessor (in the right format)
    temp = dict(
        inputs=dict(
            points=[inputs[i]['points']]    ## Must be into a list, otherwise error
        )
    )
    # Append the result to the "final_inputs" list
    final_inputs.append(preprocessor(temp))

# TODO: Add an if for the device type
for i in range(len(final_inputs)):
    for subkey in final_inputs[i]['inputs']['voxels']:
        final_inputs[i]['inputs']['voxels'][subkey] = final_inputs[i]['inputs']['voxels'][subkey].cuda()

# Print the output of this stage
if wanna_print_in_out: 
    print("\nDet3DDataPreprocessor output:")
    for element in final_inputs:
        print(element)

In [ ]:
home_dir = '/home/michele/code/'



# Import mmdeploy from the ROOT FOLDER -------> NOTE: It is not my own repository, it is the one in root (probably modified by Riccardo)
from mmdeploy.apis.inference import get_model



model = get_model(
    model_cfg= home_dir + 'michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py',
    deploy_cfg= home_dir + 'mmdeploy/configs/mmdet/detection/detection_tensorrt_dynamic-320x320-1344x1344.py',
    backend_files= ['/home/michele/code/mmdeploy/mmdeploy_models/minerva_pointpillars/end2end.engine'],
    img= home_dir + 'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235584511788288.bin',
    device= 'cuda'
)

result = model(final_inputs[0]['inputs'], final_inputs[0]['data_samples'], None)

# result = model(final_inputs[0], None, mode='predict')

# result








# print("\n\n----------------------------------------------------------")
# print(model.model_cfg.model.data_preprocessor)
# for key in model.model_cfg.model.data_preprocessor:
#     print(f"\t{key}:\n{model.model_cfg.model.data_preprocessor[key]}")
# print("\n\n----------------------------------------------------------")
# print(model.model_cfg.val_dataloader.dataset.pipeline)
# for i, element in enumerate(model.model_cfg.val_dataloader.dataset.pipeline):
#     print(f"\nElement {i}:")
#     for key in element:
#         print(f"\t{key}:\n{element[key]}")






In [ ]:
# # ############################################# TODO: Comment here #############################################
# from mmdeploy.apis.inference import inference_model
# home_dir = '/home/michele/code/'
# result = inference_model(
#     model_cfg= home_dir + 'michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py',
#     deploy_cfg= home_dir + 'mmdeploy/configs/mmdet/detection/detection_tensorrt_dynamic-320x320-1344x1344.py',
#     backend_files= ['/home/michele/code/mmdeploy/mmdeploy_models/minerva_pointpillars/end2end.engine'],
#     img= home_dir + 'michele_mmdet3d/data/minerva_polimove/training/velodyne/1723235584511788288.bin',
#     device= 'cuda'
# )



# Line 278 of object_detection_model.py:
#     def forward(self,
#                 inputs: torch.Tensor,
#                 data_samples: List[BaseDataElement],
#                 mode: str = 'predict',
#                 **kwargs) -> Any:
#         """The model forward.

#         Args:
#             inputs (torch.Tensor): The input tensors
#             data_samples (List[BaseDataElement]): The data samples.
#                 Defaults to None.
#             mode (str, optional): forward mode, only support `predict`.

#         Returns:
#             Any: Model output.
#         """
#         # assert mode == 'predict', 'Deploy model only allow mode=="predict".'
#         # inputs = inputs.contiguous()
#         outputs = self._forward(inputs, data_samples)
#         batch_dets, batch_labels = outputs[:2]
#         batch_masks = outputs[2] if len(outputs) >= 3 else None
#         self.postprocessing_results(batch_dets, batch_labels, batch_masks,
#                                     data_samples)
#         return data_samples